In [70]:
import pickle
import pandas as pd
import sys
import os
import sklearn
import json

In [ ]:
sklearn.__version__

In [34]:
year = 2023
month = 3
output_file = f'output/yellow_tripdata_{year:04d}-{month:02d}.parquet'
input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'

In [26]:
os.makedirs('output', exist_ok=True)

In [27]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [28]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [40]:
df = read_data(input_file)

In [41]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [48]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [49]:
print(f'Q1: {y_pred.std()}')

6.247488852238703


In [50]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [51]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [65]:
file_size_M = os.path.getsize(output_file) / 1024 / 1024

In [68]:
print(f'Q2: {round(file_size_M, 2)}')

Q2: 65.46


In [86]:
with open('../web-service/Pipfile.lock', 'r') as pipfile:
    scikit_hash = json.load(pipfile)["default"]["scikit-learn"]["hashes"][0]

print(scikit_hash)

sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c


In [94]:
print(f'Q1 Standard deviation for {year:04d}-{month:02d}: {y_pred.std()}')
print(f'Q2 Size of the output parquet file: {round(file_size_M, 2)}')
print(f'Q3 Command to turn the notebook into a script: jupyter nbconvert --to script starter.ipynb')
print(f'Q4 First hash for the Scikit-Learn dependency: {scikit_hash}')
print(f'Q5 Mean predicted duration for {year:04d}-{month:02d}: {round(y_pred.mean(), 2)}')

Q1 Standard deviation for 2023-03: 6.247488852238703
Q2 Size of the output parquet file: 65.46
Q3 Command to turn the notebook into a script: jupyter nbconvert --to script starter.ipynb
Q4 First hash for the Scikit-Learn dependency: sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c
Q5 Mean predicted duration for 2023-03: 14.2
